[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://githubtocolab.com/crunchdao/adialab-notebooks/blob/main/basic_submission.ipynb)

# Set up your crunch workspace

In [3]:
%pip install crunch-cli --upgrade
!crunch --notebook setup shrill-mohamed --token bRoZlvMkF2zchuhKWi4WtTq2PCfHe38a2AjwO1tiqyqcpUUnkaFPqNzlFWDn2EdF

Note: you may need to restart the kernel to use updated packages.
use .gitignore
use main.py
use requirements.txt
download data\X_train.parquet from https://datacrunch-com.s3.eu-west-1.amazonaws.com/production/adialab/data-releases/1/X_train.parquet
download data\y_train.parquet from https://datacrunch-com.s3.eu-west-1.amazonaws.com/production/adialab/data-releases/1/y_train.parquet
download data\X_test.parquet from https://datacrunch-com.s3.eu-west-1.amazonaws.com/production/adialab/data-releases/1/X_test_reduced.parquet

---
Success! Your environment has been correctly setup.
Next recommended actions:
 - To get inside your workspace directory, run: cd shrill-mohamed
 - To see all of the available commands of the CrunchDAO CLI, run: crunch --help



  0%|          | 0.00/1.71G [00:00<?, ?iB/s]
  0%|          | 164k/1.71G [00:00<20:46, 1.37MiB/s]
  0%|          | 565k/1.71G [00:00<11:28, 2.48MiB/s]
  0%|          | 999k/1.71G [00:00<11:41, 2.43MiB/s]
  0%|          | 1.93M/1.71G [00:00<06:27, 4.40MiB/s]
  0%|          | 2.42M/1.71G [00:00<06:31, 4.35MiB/s]
  0%|          | 2.88M/1.71G [00:00<06:30, 4.36MiB/s]
  0%|          | 3.34M/1.71G [00:00<06:47, 4.18MiB/s]
  0%|          | 3.78M/1.71G [00:00<06:56, 4.09MiB/s]
  0%|          | 4.25M/1.71G [00:01<06:42, 4.23MiB/s]
  0%|          | 4.99M/1.71G [00:01<05:36, 5.05MiB/s]
  0%|          | 5.51M/1.71G [00:01<05:46, 4.90MiB/s]
  0%|          | 6.00M/1.71G [00:01<05:52, 4.82MiB/s]
  0%|          | 6.50M/1.71G [00:01<05:59, 4.73MiB/s]
  0%|          | 7.00M/1.71G [00:01<06:02, 4.69MiB/s]
  0%|          | 7.64M/1.71G [00:01<05:48, 4.88MiB/s]
  0%|          | 8.13M/1.71G [00:01<06:11, 4.58MiB/s]
  1%|          | 8.60M/1.71G [00:02<06:46, 4.18MiB/s]
  1%|          | 9.04M/1.71G [00:02<06:

In [ ]:
import crunch
crunch = crunch.load_notebook(__name__)

%cd shrill-mohamed
X_train, y_train, X_test = crunch.load_data()

def temporal_train_test_split(X_train_loc, y_train_loc, test_size=0.2):
    unique_dates = X_train_loc.date.unique()
    split_date = unique_dates[int(len(unique_dates)*(1-test_size))]
    X_train_local = X_train_loc[X_train_loc['date'] <= split_date]
    X_test_local = X_train_loc[X_train_loc['date'] > split_date]
    y_train_local = y_train_loc[y_train_loc['date'] <= split_date]
    y_test_local = y_train_loc[y_train_loc['date'] > split_date]
    return X_train_local, X_test_local, y_train_local, y_test_local

print("Splitting (X_train, y_train) in X_train_local, X_test_local, y_train_local, y_test_local")
X_train_local, X_test_local, y_train_local, y_test_local = temporal_train_test_split(
    X_train,
    y_train,
    test_size=0.2
)

# First Submission

In [28]:
"""
This is a basic example of what you need to do to participate to the tournament.
The code will not have access to the internet (or any socket related operation).
"""

# Imports
import pandas as pd
import typing
import joblib
from pathlib import Path
# !pip install optuna joblib numpy pandas lightgbm scikit-learn
import optuna
import joblib
import numpy as np
import pandas as pd
import lightgbm as lightgbm
import optuna.integration.lightgbm as optuna_lgb
from collections import OrderedDict
from sklearn.model_selection import ParameterGrid

def train(X_train: pd.DataFrame, y_train: pd.DataFrame, model_directory_path: str = "resources") -> None:
    y_train.loc[:,'ranking'] = y_train.groupby("date")["y"].rank("dense", ascending=False).astype(int)
    model = lightgbm.LGBMRanker(
        objective="lambdarank",
        verbosity= -1,
        boosting_type = "gbdt",
        n_estimators = 20,
        metric= "ndcg",
        num_leaves = 400,
        learning_rate = 0.10272466691945182,
        max_depth = 35,
        label_gain =[i for i in range(y_train['ranking'].max() + 1)])
    dates_train = X_train.groupby("date")["date"].count().to_numpy()
    model.fit(
        X=X_train.iloc[:, 2:],
        y=y_train.iloc[:, 3],
        group=dates_train)
    
    #preds = X_train[['date','id']].copy()
    #preds['y'] = model.predict(X_train.iloc[:,2:])
    #preds.loc[:,'ranking'] = preds.groupby("date")["y"].rank("dense", ascending=False).astype(int)
    #y_train.loc[:,'ranking'] = y_train.groupby("date")["y"].rank("dense", ascending=False).astype(int)

    model_pathname = Path(model_directory_path) / "model.joblib"
    print(f"Saving model in {model_pathname}")
    joblib.dump(model, model_pathname)
def infer(X_test: pd.DataFrame, model_directory_path: str = "resources") -> pd.DataFrame:
    model = joblib.load(Path(model_directory_path) / "model.joblib")
    y_test_predicted = X_test[["date", "id"]].copy()
    y_test_predicted["value"] = model.predict(X_test.iloc[:, 2:])
    y_test_predicted["value"] =  y_test_predicted.groupby("date")["value"].rank("dense", ascending=False).astype(int)
    return y_test_predicted

In [29]:

from scipy.stats import spearmanr

train(X_train,y_train,'resources')
y = infer(X_test_local,"resources")
y_test_local.loc[:,'ranking'] = y_test_local.groupby("date")["y"].rank("dense", ascending=False).astype(int)
score = spearmanr(y["value"],y_test_local["ranking"])[0] * 100
print(f"Spearman's correlation {score}")



---


In [27]:
# result_eval = pd.DataFrame()
# result_eval['y'] = ranker.predict(X_eval)
# result_eval['date'] = X_eval['date'].reset_index(drop=True)
# y_result_eval = result_eval.groupby("date")["y"].rank("dense", ascending=False).astype(int)
# y_result_eval

To have a preliminary assessment of our solution above we need to create train and test data:

# Testing your submission **locally**

This function of the crunch package will run your code locally, in the same way in which the function is called in the cloud (ie: one date at a time). If it runs without problem, it is highly likely that there won't be problems when executing it on the CrunchDAO's system, on the cloud.

You can setup the a retraining frequency as you which. A train frequency of 2 means that the system will retrain your model every two dates.

`force_first_train=True` means that your model will be trained on the first date of the test set.

In [ ]:
print("Remove unused data to release memory")
del X_train, y_train, X_test, X_train_local, X_test_local, y_train_local, y_test_local

In [5]:
result = crunch.test(force_first_train=True, train_frequency=2)

16:11:06 no forbidden library found
16:11:06 
16:11:06 running local test
16:11:06 internet access isn't restricted, no check will be done
16:11:06 


download data/X_train.parquet from https://datacrunch-com.s3.eu-west-1.amazonaws.com/production/adialab/data-releases/1/X_train.parquet
already exists: file length match
download data/y_train.parquet from https://datacrunch-com.s3.eu-west-1.amazonaws.com/production/adialab/data-releases/1/y_train.parquet
already exists: file length match
download data/X_test.parquet from https://datacrunch-com.s3.eu-west-1.amazonaws.com/production/adialab/data-releases/1/X_test_reduced.parquet
already exists: file length match


16:11:12 ---
16:11:12 loop: moon=269 train=True (1/5)
16:11:12 handler: train(data/X_train.parquet, data/y_train.parquet, resources)
16:13:05 handler: infer(data/X_test.parquet, resources)
16:13:05 ---
16:13:05 loop: moon=270 train=True (2/5)
16:13:05 handler: train(data/X_train.parquet, data/y_train.parquet, resources)


Saving model in resources/model.joblib


16:16:32 handler: infer(data/X_test.parquet, resources)
16:16:32 ---
16:16:32 loop: moon=271 train=False (3/5)
16:16:32 handler: infer(data/X_test.parquet, resources)


Saving model in resources/model.joblib


16:16:32 ---
16:16:32 loop: moon=272 train=True (4/5)
16:16:32 handler: train(data/X_train.parquet, data/y_train.parquet, resources)
16:20:05 handler: infer(data/X_test.parquet, resources)


Saving model in resources/model.joblib


16:20:05 ---
16:20:05 loop: moon=273 train=False (5/5)
16:20:05 handler: infer(data/X_test.parquet, resources)
16:20:06 prediction_path=data/prediction.csv
16:20:06 duration: time=00:09:00
16:20:06 memory: before="2.97 GB" after="5.45 GB" consumed="2.48 GB"
16:20:06 local test succesfully run!


In [16]:
result

,date,id,value
0,269,c6e83eda40042dab1af117e195d542f00a417627e3173a...,1208
1,269,97ae3194605438cbd2c59a3827f7c615dafa40d6cc3f42...,94
2,269,310382927ec56f64c6f2f834fd320c9f732e26df639e67...,3146
3,269,6e963f295f4ec1dc921be47638dba304f486ab2efd313c...,1776
4,269,46cb6aa83fbd64a64a8a87d782476438abb658ce89b89c...,2752
...,...,...,...
4127,273,26e5d74e138cf23f5d65aab46fdc59f6421e97ccf1ab8e...,1088
4128,273,9dd2e69f186ef4eb076c646a8b182e936af2667793d143...,360
4129,273,002647639e3b83fd884eed0eddf72a702f15c5d70fb75d...,593
4130,273,7d840103d2370a80cc9b8376bfaf04b2aa5ff46bcbab03...,802


# Now remember to download and then submit this notebook to https://adialab.crunchdao.com/submit